In [1]:
import cv2 as cv
import numpy as np


hsv = 0
lower_blue1 = 0
upper_blue1 = 0
lower_blue2 = 0
upper_blue2 = 0
lower_blue3 = 0
upper_blue3 = 0


def nothing(x):
    pass

def mouse_callback(event, x, y, flags, param):
    global hsv, lower_blue1, upper_blue1, lower_blue2, upper_blue2, lower_blue3, upper_blue3, threshold

    # 마우스 왼쪽 버튼 누를시 위치에 있는 픽셀값을 읽어와서 HSV로 변환합니다.
    if event == cv.EVENT_LBUTTONDOWN:
        print(img_color[y, x])
        color = img_color[y, x]

        one_pixel = np.uint8([[color]])
        hsv = cv.cvtColor(one_pixel, cv.COLOR_BGR2HSV)
        hsv = hsv[0][0]

        threshold = cv.getTrackbarPos('threshold', 'img_result')
        
        # HSV 색공간에서 마우스 클릭으로 얻은 픽셀값과 유사한 필셀값의 범위를 정합니다.
        if hsv[0] < 10:
            print("case1")
            lower_blue1 = np.array([hsv[0]-10+180, threshold, threshold])
            upper_blue1 = np.array([180, 255, 255])
            lower_blue2 = np.array([0, threshold, threshold])
            upper_blue2 = np.array([hsv[0], 255, 255])
            lower_blue3 = np.array([hsv[0], threshold, threshold])
            upper_blue3 = np.array([hsv[0]+10, 255, 255])
            #     print(i-10+180, 180, 0, i)
            #     print(i, i+10)

        elif hsv[0] > 170:
            print("case2")
            lower_blue1 = np.array([hsv[0], threshold, threshold])
            upper_blue1 = np.array([180, 255, 255])
            lower_blue2 = np.array([0, threshold, threshold])
            upper_blue2 = np.array([hsv[0]+10-180, 255, 255])
            lower_blue3 = np.array([hsv[0]-10, threshold, threshold])
            upper_blue3 = np.array([hsv[0], 255, 255])
            #     print(i, 180, 0, i+10-180)
            #     print(i-10, i)
        else:
            print("case3")
            lower_blue1 = np.array([hsv[0], threshold, threshold])
            upper_blue1 = np.array([hsv[0]+10, 255, 255])
            lower_blue2 = np.array([hsv[0]-10, threshold, threshold])
            upper_blue2 = np.array([hsv[0], 255, 255])
            lower_blue3 = np.array([hsv[0]-10, threshold, threshold])
            upper_blue3 = np.array([hsv[0], 255, 255])
            #     print(i, i+10)
            #     print(i-10, i)

        print(hsv[0])
        print("@1", lower_blue1, "~", upper_blue1)
        print("@2", lower_blue2, "~", upper_blue2)
        print("@3", lower_blue3, "~", upper_blue3)


cv.namedWindow('img_color')
cv.setMouseCallback('img_color', mouse_callback)

cv.namedWindow('img_result')
cv.createTrackbar('threshold', 'img_result', 0, 255, nothing)
cv.setTrackbarPos('threshold', 'img_result', 30)

cap = cv.VideoCapture(0)

while(True):
    #img_color = cv.imread('2.jpg')
    ret,img_color = cap.read()
    height, width = img_color.shape[:2]
    img_color = cv.resize(img_color, (width, height), interpolation=cv.INTER_AREA)

    # 원본 영상을 HSV 영상으로 변환합니다.
    img_hsv = cv.cvtColor(img_color, cv.COLOR_BGR2HSV)

    # 범위 값으로 HSV 이미지에서 마스크를 생성합니다.
    img_mask1 = cv.inRange(img_hsv, lower_blue1, upper_blue1)
    img_mask2 = cv.inRange(img_hsv, lower_blue2, upper_blue2)
    img_mask3 = cv.inRange(img_hsv, lower_blue3, upper_blue3)
    img_mask = img_mask1 | img_mask2 | img_mask3

    kernel = np.ones((11,11), np.uint8)
    img_mask = cv.morphologyEx(img_mask, cv.MORPH_OPEN, kernel)
    img_mask = cv.morphologyEx(img_mask, cv.MORPH_CLOSE, kernel)

    # 마스크 이미지로 원본 이미지에서 범위값에 해당되는 영상 부분을 획득합니다.
    img_result = cv.bitwise_and(img_color, img_color, mask=img_mask)


    numOfLabels, img_label, stats, centroids = cv.connectedComponentsWithStats(img_mask)

    for idx, centroid in enumerate(centroids):
        if stats[idx][0] == 0 and stats[idx][1] == 0:
            continue

        if np.any(np.isnan(centroid)):
            continue

        x,y,width,height,area = stats[idx]
        centerX,centerY = int(centroid[0]), int(centroid[1])
        #print(centerX, centerY)
        print(width, height)
        print(height)

        if area > 50:
            cv.circle(img_color, (centerX, centerY), 10, (0,0,255), 10)
            cv.rectangle(img_color, (x,y), (x+width,y+height), (0,0,255))

    cv.imshow('img_color', img_color)
    cv.imshow('img_mask', img_mask)
    cv.imshow('img_result', img_result)


    # ESC 키누르면 종료
    if cv.waitKey(1) & 0xFF == 27:
        break

[201 224 224]
case3
30
@1 [30 30 30] ~ [ 40 255 255]
@2 [20 30 30] ~ [ 30 255 255]
@3 [20 30 30] ~ [ 30 255 255]
36 15
7.5
24 22
11.0
28 88
44.0
42 28
14.0
54 15
7.5
12 12
6.0
50 28
14.0
26 83
41.5
28 51
25.5
18 21
10.5
27 14
7.0
55 65
32.5
22 23
11.5
26 77
38.5
17 16
8.0
12 11
5.5
26 14
7.0
12 11
5.5
52 61
30.5
18 21
10.5
26 75
37.5
30 15
7.5
12 11
5.5
54 45
22.5
24 29
14.5
26 81
40.5
40 14
7.0
23 26
13.0
46 38
19.0
26 81
40.5
20 13
6.5
26 14
7.0
22 25
12.5
51 45
22.5
26 88
44.0
14 12
6.0
40 18
9.0
12 14
7.0
24 25
12.5
55 43
21.5
26 84
42.0
12 11
5.5
13 11
5.5
26 13
6.5
12 11
5.5
52 43
21.5
24 18
9.0
26 77
38.5
14 11
5.5
12 11
5.5
16 12
6.0
26 70
35.0
56 44
22.0
11 11
5.5
12 11
5.5
26 12
6.0
54 64
32.0
24 22
11.0
26 25
12.5
26 85
42.5
12 11
5.5
22 13
6.5
56 67
33.5
12 11
5.5
26 78
39.0
11 12
6.0
6 10
5.0
26 14
7.0
12 12
6.0
58 48
24.0
22 21
10.5
26 81
40.5
13 13
6.5
16 13
6.5
14 11
5.5
20 98
49.0
21 23
11.5
52 49
24.5
12 11
5.5
14 12
6.0
16 34
17.0
52 44
22.0
26 93
46.5
17 17
8.5
24 1

40 24
12.0
64 103
51.5
74 198
99.0
24 16
8.0
6 11
5.5
22 12
6.0
104 130
65.0
72 88
44.0
38 22
11.0
58 101
50.5
6 12
6.0
12 12
6.0
15 14
7.0
70 83
41.5
81 77
38.5
12 11
5.5
13 12
6.0
70 111
55.5
6 14
7.0
112 114
57.0
78 38
19.0
18 36
18.0
12 15
7.5
80 128
64.0
12 11
5.5
70 54
27.0
82 206
103.0
16 23
11.5
12 11
5.5
20 18
9.0
16 15
7.5
74 84
42.0
58 77
38.5
8 32
16.0
56 117
58.5
11 11
5.5
12 12
6.0
28 18
9.0
104 212
106.0
58 23
11.5
12 13
6.5
12 11
5.5
20 30
15.0
16 17
8.5
14 17
8.5
21 25
12.5
96 215
107.5
70 90
45.0
12 16
8.0
8 11
5.5
18 14
7.0
13 6
3.0
96 234
117.0
44 88
44.0
12 16
8.0
6 15
7.5
12 15
7.5
6 16
8.0
96 234
117.0
44 88
44.0
12 16
8.0
6 15
7.5
12 15
7.5
6 16
8.0
12 11
5.5
102 75
37.5
54 57
28.5
84 113
56.5
6 11
5.5
106 206
103.0
52 38
19.0
16 22
11.0
12 11
5.5
16 7
3.5
11 12
6.0
100 211
105.5
16 16
8.0
14 15
7.5
8 14
7.0
16 11
5.5
102 93
46.5
32 20
10.0
6 14
7.0
66 107
53.5
14 12
6.0
100 94
47.0
40 36
18.0
12 15
7.5
13 11
5.5
38 22
11.0
62 74
37.0
101 76
38.0
14 18
9.0
89 12

104 100
50.0
60 28
14.0
54 26
13.0
104 100
50.0
24 21
10.5
62 23
11.5
104 101
50.5
42 22
11.0
59 22
11.0
106 101
50.5
56 25
12.5
57 23
11.5
106 100
50.0
72 27
13.5
61 22
11.0
104 100
50.0
68 27
13.5
56 25
12.5
104 99
49.5
78 30
15.0
62 24
12.0
106 98
49.0
46 19
9.5
62 21
10.5
106 99
49.5
68 23
11.5
62 23
11.5
104 99
49.5
75 25
12.5
59 23
11.5
105 99
49.5
70 31
15.5
61 26
13.0
105 100
50.0
78 29
14.5
62 24
12.0
106 101
50.5
78 28
14.0
63 22
11.0
106 99
49.5
78 28
14.0
66 27
13.5
104 101
50.5
75 33
16.5
60 23
11.5
104 99
49.5
78 29
14.5
52 24
12.0
104 100
50.0
77 30
15.0
63 25
12.5
104 99
49.5
77 32
16.0
64 24
12.0
106 99
49.5
68 30
15.0
58 20
10.0
104 98
49.0
69 25
12.5
60 22
11.0
106 99
49.5
56 30
15.0
60 24
12.0
104 99
49.5
52 25
12.5
60 22
11.0
104 100
50.0
42 22
11.0
60 24
12.0
104 100
50.0
38 24
12.0
58 21
10.5
104 99
49.5
42 26
13.0
13 16
8.0
52 22
11.0
104 100
50.0
50 29
14.5
52 23
11.5
104 101
50.5
72 21
10.5
64 22
11.0
104 101
50.5
20 20
10.0
22 20
10.0
58 23
11.5
104 100
50.0


27 16
8.0
102 99
49.5
22 15
7.5
102 98
49.0
46 17
8.5
102 99
49.5
48 20
10.0
102 99
49.5
44 15
7.5
102 98
49.0
46 16
8.0
102 98
49.0
102 99
49.5
44 17
8.5
102 98
49.0
29 19
9.5
102 98
49.0
46 18
9.0
102 97
48.5
34 14
7.0
12 11
5.5
102 98
49.0
48 19
9.5
12 11
5.5
102 98
49.0
48 19
9.5
102 98
49.0
40 17
8.5
102 98
49.0
46 19
9.5
11 11
5.5
102 98
49.0
60 21
10.5
102 96
48.0
56 21
10.5
102 97
48.5
30 27
13.5
102 97
48.5
24 21
10.5
18 11
5.5
102 97
48.5
40 21
10.5
102 97
48.5
11 11
5.5
16 13
6.5
100 98
49.0
24 24
12.0
22 20
10.0
102 98
49.0
28 15
7.5
16 21
10.5
102 99
49.5
32 20
10.0
18 11
5.5
18 16
8.0
100 99
49.5
48 19
9.5
18 11
5.5
18 16
8.0
100 99
49.5
48 19
9.5
102 99
49.5
100 98
49.0
102 99
49.5
14 13
6.5
102 99
49.5
102 100
50.0
100 99
49.5
16 15
7.5
102 97
48.5
100 98
49.0
12 12
6.0
100 97
48.5
100 98
49.0
102 98
49.0
102 98
49.0
13 13
6.5
100 98
49.0
102 98
49.0
14 13
6.5
102 97
48.5
14 12
6.0
100 98
49.0
16 13
6.5
102 98
49.0
102 98
49.0
100 98
49.0
102 98
49.0
102 98
49.0
15 16
8

31 15
7.5
20 15
7.5
34 16
8.0
34 16
8.0
32 18
9.0
36 17
8.5
44 19
9.5
42 19
9.5
40 19
9.5
44 19
9.5
41 20
10.0
46 28
14.0
46 28
14.0
52 29
14.5
51 30
15.0
46 28
14.0
52 28
14.0
50 31
15.5
48 30
15.0
54 28
14.0
51 29
14.5
54 29
14.5
56 30
15.0
60 27
13.5
48 27
13.5
56 27
13.5
54 28
14.0
50 25
12.5
50 28
14.0
50 28
14.0
50 28
14.0
52 27
13.5
58 30
15.0
54 27
13.5
60 31
15.5
63 28
14.0
62 30
15.0
55 30
15.0
62 32
16.0
58 26
13.0
63 30
15.0
61 28
14.0
59 30
15.0
64 28
14.0
59 31
15.5
53 32
16.0
57 28
14.0
61 28
14.0
50 24
12.0
12 13
6.5
58 26
13.0
18 14
7.0
62 26
13.0
44 25
12.5
60 28
14.0
42 31
15.5
60 29
14.5
22 34
17.0
55 27
13.5
22 34
17.0
55 27
13.5
44 33
16.5
56 26
13.0
62 21
10.5
50 27
13.5
34 24
12.0
60 32
16.0
40 35
17.5
58 29
14.5
36 26
13.0
52 29
14.5
26 25
12.5
58 29
14.5
19 18
9.0
59 26
13.0
55 29
14.5
60 28
14.0
[167 120 136]
case3
130
@1 [130  53  53] ~ [140 255 255]
@2 [120  53  53] ~ [130 255 255]
@3 [120  53  53] ~ [130 255 255]
114 123
61.5
114 123
61.5
114 121
60.5
115 

112 112
56.0
114 112
56.0
114 113
56.5
114 113
56.5
112 112
56.0
112 112
56.0
112 113
56.5
114 111
55.5
114 113
56.5
112 111
55.5
111 111
55.5
112 110
55.0
112 112
56.0
112 111
55.5
110 111
55.5
110 111
55.5
110 111
55.5
110 111
55.5
110 110
55.0
110 111
55.5
110 111
55.5
112 111
55.5
110 111
55.5
110 111
55.5
110 110
55.0
110 111
55.5
110 111
55.5
110 111
55.5
110 111
55.5
110 111
55.5
110 110
55.0
110 111
55.5
110 112
56.0
110 111
55.5
110 111
55.5
108 111
55.5
108 111
55.5
110 112
56.0
110 112
56.0
110 112
56.0
110 112
56.0
110 111
55.5
110 111
55.5
110 111
55.5
110 110
55.0
108 110
55.0
108 111
55.5
108 110
55.0
108 110
55.0
108 110
55.0
110 110
55.0
110 111
55.5
110 111
55.5
109 111
55.5
110 109
54.5
110 111
55.5
108 110
55.0
110 111
55.5
110 111
55.5
109 110
55.0
110 110
55.0
110 110
55.0
108 110
55.0
108 109
54.5
108 111
55.5
108 111
55.5
109 110
55.0
108 109
54.5
108 110
55.0
108 110
55.0
108 110
55.0
108 111
55.5
110 111
55.5
110 110
55.0
108 110
55.0
110 111
55.5
110 109
54.5